## Github Link: [https://github.com/JacobHurley/Final-Project](https://github.com/JacobHurley/Final-Project)

# Introduction

## Why Spotify data?
For a few years now I have it hard to discover new music even though I use platforms like spotify. I think it is important for people to continue to find new music to keep their lives interesting. Music can change a person's emotion so ensuring that they are happy with new music is important.

## How do we find new music?
Everyone has a taste when it comes to music and typically they can define it by genres that they like more than others. Other people may hear a new song in a group of friends and begin to enjoy and listen to it themselves. In our case, we will solve the problem using the prior and recomending songs similar to the user's favorite (most listened to) songs. This will be done by using an unsupervised recomendation algorithm with a item-item similarity matrix.

## Where did the data come from?
This data was pulled from multiple different datasets and spotify, and combined at [https://www.kaggle.com/datasets/undefinenull/million-song-dataset-spotify-lastfm/data](https://www.kaggle.com/datasets/undefinenull/million-song-dataset-spotify-lastfm/data)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
import warnings
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import os
import time
from sklearn.decomposition import NMF, TruncatedSVD

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [3]:
songData = pd.read_csv("Data/Music Info.csv")
userData = pd.read_csv("Data/User Listening History.csv")

In [4]:
display(songData.head(5))
display(userData.head(5))
print("Song Data shape: {}".format(songData.shape))
print("user Data shape: {}".format(userData.shape))

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,1,-4.360,1,0.0746,0.001190,0.000000,0.0971,0.240,148.114,4
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,2,-4.373,1,0.0336,0.000807,0.000000,0.2070,0.651,174.426,4
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,4,-5.783,0,0.0400,0.000175,0.000459,0.0878,0.543,120.012,4
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,9,-8.851,1,0.0371,0.000389,0.000655,0.1330,0.490,104.560,4
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,7,-9.935,1,0.0369,0.010200,0.000141,0.1290,0.104,91.841,4


,track_id,user_id,playcount
0,TRIRLYL128F42539D1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
1,TRFUPBA128F934F7E1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
2,TRLQPQJ128F42AA94F,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
3,TRTUCUY128F92E1D24,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1
4,TRHDDQG12903CB53EE,b80344d063b5ccb3212f76538f3d9e43d87dca9e,1


Song Data shape: (50683, 21)
user Data shape: (9711301, 3)


## Data Information
We have created 2 dataframe objects, one for the userListening record, and one for the songInformation record. 
The user dataframe has only 3 columns (track_id, user_id, and playcount). This relationship shows the number of times a user has listened to a specific song stored by track_id. It has over 9 million rows which will become a problem later.
The Song dataframe has 21 columns in it including 3 categorical columns such as artist, tags, and genre. It also includes many continuous columns such as duration_ms, danceability, loudness, and more. It also contains 50,683 rows which are the number of unique songs. The only columns that contains nan values in either dataframe is the tags column and the genre column.

## Data Cleaning part 1
First, we will start by updating the song dataframe into something that can be processed mathematically. We will begin by creating a function to split the tags and create a new column for each of them. This is because most of the tags listed are similar to genres and this way we can use a jaccard_similarity function on the song tags. If they stayed as their string format it would be hard to tell if 2 songs were similar based on the text.

In [7]:
# split tags into list
def splitFunc(x):
    return str(x).split(", ")
temp = songData['tags'].apply(splitFunc)
print(temp)

0        [rock, alternative, indie, alternative_rock, i...
1        [rock, alternative, indie, pop, alternative_ro...
2        [rock, alternative, alternative_rock, 90s, gru...
3        [rock, alternative, indie, alternative_rock, i...
4        [rock, alternative, indie, alternative_rock, i...
                               ...                        
50678                                                [nan]
50679            [rock, alternative_rock, japanese, cover]
50680                    [metal, metalcore, post_hardcore]
50681                                                [nan]
50682                                     [rock, japanese]
Name: tags, Length: 50683, dtype: object


In [8]:
uniqueTags = set()
for tags in temp:
    for tag in tags:
        uniqueTags.add(tag.lower())
#print(uniqueTags)
print("number of unique tags: {}".format(len(uniqueTags)))

number of unique tags: 101


We see that there are 101 unique tags found from the tags column including a 'nan' tag. We will keep this tag as a column because songs that dont have a genre or tag may be classified as similar.

In [10]:
tempSongs = songData.copy()
for row in range(0, len(temp)):
    for column in temp[row]:
        tempSongs.loc[tempSongs.index[row], column] = 1 # If a song has the tag, put a 1 in the associated column
display(tempSongs)

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,new_age,black_metal,death_metal,classical,japanese,melodic_death_metal,grindcore,j_pop,polish,nan
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,"rock, alternative, indie, alternative_rock, in...",NaN,2004,222200,0.355,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,"rock, alternative, indie, pop, alternative_roc...",NaN,2006,258613,0.409,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,"rock, alternative, alternative_rock, 90s, grunge",RnB,1991,218920,0.508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,"rock, alternative, indie, alternative_rock, in...",NaN,2004,237026,0.279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,"rock, alternative, indie, alternative_rock, in...",RnB,2008,238640,0.515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50678,TRQYCFV128F9322F50,Ryusei Rocket,アンティック-珈琲店-,https://p.scdn.co/mp3-preview/d2668a5a3e0b1fda...,0tt1RdeJX1RyuU4hMEZ19T,NaN,NaN,2008,273440,0.438,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
50679,TRHQCSH128F42724B7,Colors Of The Wind,ACIDMAN,https://p.scdn.co/mp3-preview/8e22a7052ef3ecf7...,3wkdfXGf5JYErW4b35zP2h,"rock, alternative_rock, japanese, cover",NaN,2004,275133,0.351,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
50680,TRZRODK128F92D68D7,The Revelation,coldrain,https://p.scdn.co/mp3-preview/4b51a813f67e3853...,1gXMORZRGA40PE9rDE9cja,"metal, metalcore, post_hardcore",NaN,2014,254826,0.434,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50681,TRGLMEM128F9322F63,Koi no Dependence,アンティック-珈琲店-,https://p.scdn.co/mp3-preview/5a61e031df174666...,1dxMDGvIYHFYgRvmw1uMHG,NaN,NaN,2008,243293,0.513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


After creating the 101 new columns, rows that did not have the associated tag for the column were populated with nan values. We must fix this by filling them all in with 0 where it is na to represent the song not having that tag.

In [12]:
# cell to get rid of NAN values in our newly created columns
tempSongs[list(uniqueTags)] = tempSongs[list(uniqueTags)].fillna(0)

In [13]:
"""# cell for checking if values are updated correctly
newTemp = tempSongs.columns
for column in tempSongs.columns:
    print("{} : {}".format(column, tempSongs.loc[tempSongs.index[0], column]))"""

'# cell for checking if values are updated correctly\nnewTemp = tempSongs.columns\nfor column in tempSongs.columns:\n    print("{} : {}".format(column, tempSongs.loc[tempSongs.index[0], column]))'

once our new columns are added, we can get rid of the tags column because it does not provide us any new information and can not be properly processed by our algorithm.

In [15]:
tempSongs = tempSongs.drop(columns=['tags'])

Next, lets again check to see which columns have na values and how many there are.

In [17]:
# check for NA values in each column
for column in tempSongs.columns:
    NAcount = tempSongs[column].isna().sum()
    if(NAcount > 0):
        print("{} : {}".format(column, NAcount))
print("Songs DF shape: {}".format(tempSongs.shape))

genre : 28335
Songs DF shape: (50683, 121)


We are now going to check to see what the unique values are in the genre column and if they overlap with the tags we already saw. We make sure to set the string to lower since all of the tags were lower case.

In [19]:
uniqueGenres = set(str(x).lower() for x in tempSongs['genre'].unique())
#print("Unique Genres: {}".format(sorted(uniqueGenres)))
#print("Unique Tags: {}".format(sorted(uniqueTags)))
uniqueGenres- uniqueTags

{'latin', 'new age', 'world'}

Because more than half of the genre column are null and they are just repeats of tags when applicable, we will ensure that the tag column is set to 1 or add a new column and delete the genre column after. Note that the 'new age' genre is already a tag called 'new_age' 

In [21]:
for row in range(0, tempSongs.shape[0]):
    if (str(tempSongs.loc[tempSongs.index[row], 'genre']).lower() == 'new age'):
        tempSongs.loc[tempSongs.index[row], 'new_age'] = 1
    else:
        tempSongs.loc[tempSongs.index[row], str(tempSongs.loc[tempSongs.index[row], 'genre']).lower()] = 1

In [22]:
print("Songs DF shape: {}".format(tempSongs.shape))

Songs DF shape: (50683, 123)


In [23]:
#display(tempSongs)

As we did earlier with the columns created from tags, we must make sure to fill in the nan values from our newly added columns with 0 and then drop our 'genre' column

In [25]:
tempSongs[['latin','world']] = tempSongs[['latin','world']].fillna(0)

In [26]:
tempSongs = tempSongs.drop(columns=['genre'])

For the last part of our Data Cleaning, we will convert the duration_ms column into duration_seconds rounded. This is because there is a lot of variablility in milliseconds and we do not want the cosine similarity operation to say that 2 songs are different because they are not the same time in milliseconds. We will also change the name of the column when we do this to avoid confusion

In [28]:
def MStoS(x):
    return round(x/1000)

tempSongs['duration_ms'] = tempSongs['duration_ms'].apply(MStoS)
tempSongs = tempSongs.rename(columns={"duration_ms": "duration_seconds"})

In [29]:
"""for column in sorted(tempSongs.columns):
    print(column)
# this is a statement to show all the song columns and is not needed anymore"""

'for column in sorted(tempSongs.columns):\n    print(column)\n# this is a statement to show all the song columns and is not needed anymore'

In [30]:
display(tempSongs)

,track_id,name,artist,spotify_preview_url,spotify_id,year,duration_seconds,danceability,energy,key,...,death_metal,classical,japanese,melodic_death_metal,grindcore,j_pop,polish,nan,latin,world
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,09ZQ5TmUG8TSL56n0knqrj,2004,222,0.355,0.918,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,06UfBBDISthj1ZJAtX4xjj,2006,259,0.409,0.892,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,TROUVHL128F426C441,Come as You Are,Nirvana,https://p.scdn.co/mp3-preview/a1c11bb1cb231031...,0keNu0t0tqsWtExGM3nT1D,1991,219,0.508,0.826,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TRUEIND128F93038C4,Take Me Out,Franz Ferdinand,https://p.scdn.co/mp3-preview/399c401370438be4...,0ancVQ9wEcHVd0RrGICTE4,2004,237,0.279,0.664,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,TRLNZBD128F935E4D8,Creep,Radiohead,https://p.scdn.co/mp3-preview/e7eb60e9466bc3a2...,01QoK9DA7VTeTSE3MNzp4I,2008,239,0.515,0.430,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50678,TRQYCFV128F9322F50,Ryusei Rocket,アンティック-珈琲店-,https://p.scdn.co/mp3-preview/d2668a5a3e0b1fda...,0tt1RdeJX1RyuU4hMEZ19T,2008,273,0.438,0.933,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
50679,TRHQCSH128F42724B7,Colors Of The Wind,ACIDMAN,https://p.scdn.co/mp3-preview/8e22a7052ef3ecf7...,3wkdfXGf5JYErW4b35zP2h,2004,275,0.351,0.693,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
50680,TRZRODK128F92D68D7,The Revelation,coldrain,https://p.scdn.co/mp3-preview/4b51a813f67e3853...,1gXMORZRGA40PE9rDE9cja,2014,255,0.434,0.975,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
50681,TRGLMEM128F9322F63,Koi no Dependence,アンティック-珈琲店-,https://p.scdn.co/mp3-preview/5a61e031df174666...,1dxMDGvIYHFYgRvmw1uMHG,2008,243,0.513,0.902,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [31]:
print(len(tempSongs['artist'].unique()))
''' Too many artists to create a column for each, will base the recomendation system off similarity of music rather than artist
'''

8317


' Too many artists to create a column for each, will base the recomendation system off similarity of music rather than artist\n'

After checking how many unique artists there are for music in our dataframe, it will be too much to create a new column for every single one. We will use the assumption however that music produced by the same artist will have similar features elsewhere such as year, all the tag/genre columns, and many of the song descriptions such as energy and danceability.

## Data Cleaning part 2
Now that the song dataFrame is cleaned up, we must clean up the userData dataFrame. Luckily there are no nan values in this dataframe and the columns are already correct and do not need to be expanded, however there are too many rows and we can not fit this amount of data in a matrix. The way we will fix this is by getting rid of users that have less than 30 total playcount.

In [34]:
print(len(userData['user_id'].unique()))

962037


In [35]:
# create new DF where row index is the user_id, and column index is the track_id

#combined = pd.DataFrame(index = sorted(userData['user_id'].unique()), columns = sorted(tempSongs['track_id'].unique()), dtype=np.uint16)
#display(combined)




In [36]:
UsersPlaycount = userData.groupby(['user_id'])['playcount'].sum()

We will use an arbitrary cut-off of 30 songs listened to to determine who to include in our dataframe. This does not mean that they need to listen to 30 different songs, but could include the same song 30 times.

In [38]:
userplaycountminimum = 30
UsersPlaycount = UsersPlaycount[UsersPlaycount >= userplaycountminimum]

As we can see, we reduced our unique users by about 700 thousand by applying this filter. This will make our dataframe smaller and able to run on RAM

In [40]:
uniqueUsersBefore = len(userData['user_id'].unique())
userData = userData[userData['user_id'].isin(UsersPlaycount.keys())]

In [41]:
"""
# this cell is just to display an individual's userData
userData[userData['user_id'] == 'fffff9534445f481b6ab91c345500083d2ce4df1']
"""

"\n# this cell is just to display an individual's userData\nuserData[userData['user_id'] == 'fffff9534445f481b6ab91c345500083d2ce4df1']\n"

## Data cleaning Final part
We can now create our combined matrix where the rows are the users and the columns are the songs. We will use user_id for users and track_id for songs and store the values as unsigned 8bit integers. We chose unsigned 16bit integers because they can still represent a very large value but take up much less storage space than a 32bit variable.

In [43]:
combinedMatrix = coo_matrix((len(userData['user_id'].unique()), len(tempSongs['track_id'].unique())), dtype=np.uint8).toarray()

We wil create some dictionaries to store the index for each user and song in the combined matrix as well as dictionaries to reverse-map the index to the value

In [45]:
id2index = {value : index for index, value in enumerate(sorted(userData['user_id'].unique()))}
# id2index['000138e252eea35fd73aaf66a9b34102b695a9c8'] == 1

In [46]:
song2index = {value : index for index, value in enumerate(sorted(tempSongs['track_id'].unique()))}

In [47]:
index2song = dict((v,k) for k,v in song2index.items())
index2user = dict((v,k) for k,v in id2index.items())
trackIDtoTitle = tempSongs.set_index('track_id')['name'].to_dict()

In [48]:
#song2index['TRAAAED128E0783FAB'] == 1

In [49]:
# tempSongs[tempSongs['track_id'] == 'TRAAADT12903CCC339']

Next we will fill the combined matrix with the percentage that the user listens to an individual song. This will cause it to have a greater weight towards songs that fill up the user's playcount, but when the user begins to listen to many songs an almost equal amount, a single song being listened to one more time than another will not have as large of an impact on reccomendations. This also allows us to use unsigned 8bit integer values instead as well since the input will range from 0 to 100.

In [51]:
for index, row in userData.iterrows():
    combinedMatrix[id2index[row['user_id']]][song2index[row['track_id']]] = round(row['playcount'] / UsersPlaycount[row['user_id']]) * 100
# dividing by the total user playcount to bring the values closer together

In [52]:
# this is a check to make sure values were updated properly, should equal 20 if worked properly or different value if we scale
# print(combinedMatrix[id2index['fffff9534445f481b6ab91c345500083d2ce4df1']][song2index['TRLHPYP128F4259AB7']])

We can make our song-song similarity matrix computation simpler if we sort the values of the track_id column first.

In [54]:
newTemp = tempSongs.sort_values(by='track_id')
#display(newTemp)

We can now drop columns that contain bad values such as links, name of song, track_id (because we already have it stored using the index), and artist name since we did not want to one-hot encode all the unique values.

In [56]:
newTemp = newTemp.drop(columns=['track_id', 'name', 'artist', 'spotify_preview_url', 'spotify_id'])
display(newTemp)

,year,duration_seconds,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,death_metal,classical,japanese,melodic_death_metal,grindcore,j_pop,polish,nan,latin,world
9639,1997,219,0.714,0.521,2,-9.828,1,0.0363,0.72700,0.825000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9086,2004,247,0.565,0.507,9,-8.339,1,0.0541,0.70200,0.000066,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6113,2011,242,0.548,0.843,2,-2.674,1,0.0374,0.04670,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4625,2003,217,0.720,0.643,10,-7.805,1,0.1000,0.58000,0.072800,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
17366,2009,182,0.908,0.695,2,-7.680,0,0.0448,0.11800,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37697,1999,265,0.588,0.835,9,-5.497,1,0.0356,0.12100,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37421,2006,222,0.172,0.948,0,-4.194,1,0.1800,0.00173,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40155,2001,365,0.487,0.601,11,-12.319,1,0.0327,0.94700,0.889000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
18144,2004,430,0.485,0.452,2,-10.250,1,0.0274,0.12500,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Exploratory Data Analysis
We will now look at some of the data, like correlation matrices for our music data

In [58]:
fig, ax = plt.subplots(figsize=(100,100))
sns.heatmap(newTemp.corr(), annot=True, fmt='.1f')

<Axes: >

It is hard to tell, but if you zoom into the correlation matrix, you can see a few values with a correlation of about .7 or close to it. These categories are hip_hop and rap, and punk and punk_rock.

## Models
We should also have a function that will display the rows for the songs that we recomend to the user. In this case we will use the entire row, however in production it would either play the next song, show the link, or show the artist name and song name rather than the entire row

In [61]:
def showRecomendations(recomendations):
    for recomendation in recomendations:
        display(songData[songData['name'] == trackIDtoTitle[index2song[recomendation]]])

## Item-Item Similarity Model
We will also need a function that that fastly calculate the jaccard similarity between a matrix1 and a matrix2 where matrix 1 is a subset of matrix2. Matrix1 will be a small amount of songs compared to matrix2 which will be all songs.

In [63]:
# this code was obtained as a prompt from chatgpt to make finding the jaccard similarity faster
def jaccard_sim_vectorized(matrix1, matrix2):
    matrix1 = matrix1.values
    matrix2 = matrix2.values
    intersection = np.dot(matrix1, matrix2.T)
    union = np.add(
        np.sum(matrix1, axis=1)[:, None],
        np.sum(matrix2, axis=1)[None, :]
    ) - intersection
    with np.errstate(divide='ignore', invalid='ignore'):
        similarity_matrix = np.divide(intersection, union)
        similarity_matrix[union == 0] = 0
    return similarity_matrix

We will need a large matrix to store the song-song similarity matrix where it is of size (songs, songs) which is about (50000,500000) and will have to store 32bit float values. This is a large amount of space that can not be done with only RAM, and so we will take advantage of numpy being able to map memory to a disk and read it fast. We will also use cosine similarity for continuous data and take tha jaccard similarity for our tags columns and average the 2 together in order to produce a suedo-similarity that we can use. The weight of each similarity may change as we update the algorithm.

In [65]:
# method for using np.memmap to store cosine similarity matrix found using google and it's AI answer

songSimilarityFile = "songSimilarity.dat"
similarity_shape = (tempSongs.shape[0], tempSongs.shape[0])
similarity_dtype = np.float32
if os.path.exists(songSimilarityFile): #because we have ran this code already and it takes a bit, we will load it in since it is already saved to memory
    similarity_memmap = np.memmap(songSimilarityFile, dtype=similarity_dtype, mode='r', shape=similarity_shape)
    #display(similarity_memmap)
else:
    similarity_memmap = np.memmap(songSimilarityFile, dtype=similarity_dtype, mode='w+', shape=similarity_shape)

    chunk_size = 1000
    for row in range(0, tempSongs.shape[0], chunk_size):
        S = cosine_similarity(newTemp.iloc[row: row + chunk_size, 0:14], newTemp.iloc[:,0:14])
        J = jaccard_sim_vectorized(newTemp.iloc[row: row + chunk_size, 14:], newTemp.iloc[:, 14:])
        retval = (S+J)/2
        similarity_memmap[row:row + chunk_size] = retval

    similarity_memmap.flush()
    del similarity_memmap
    similarity_memmap = np.memmap(songSimilarityFile, dtype=similarity_dtype, mode='r', shape=similarity_shape)
    #display(similarity_memmap)

We need a function that will compute the dot product from the userID to the similarity matrix and return the top scores (recomended songs). In this case, the algorithm can still recomend the most listened to song, however we can change this by simply excluding from our search and enforcing that it can't be one of their top N songs. This would change depending on if we are queuing a song automatically and think they would like to listen to their most favorite song again or if they are trying to discover new music.

In [67]:
# this method uses songs that the user has listened to and provides songs that are similar to those songs.
def predictSongsFromUser(userID, n):
    recItems = np.dot(combinedMatrix[id2index[userID]], similarity_memmap)
    retval = np.argpartition(recItems, -n)[-n:]
    return retval

predictions = predictSongsFromUser('fffff9534445f481b6ab91c345500083d2ce4df1', 4)
showRecomendations(predictions)

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
13890,TRCKGWO12903CBC4DD,Real Wild Child,Iggy Pop,https://p.scdn.co/mp3-preview/5f2f01f78b362af9...,00D2x2aCgoqo1AsnmcpdSI,"rock, punk, 80s, punk_rock, new_wave, post_punk",NaN,2014,213520,0.338,...,9,-12.014,1,0.0549,0.000922,0.257,0.946,0.352,147.492,4


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
28480,TRAICJY128F92C3FF2,My Perfect Cousin,The Undertones,https://p.scdn.co/mp3-preview/ee4e6cfbfb9ea6c1...,1DxNV3CnFJdsUSRoHAbDAw,"punk, 80s, punk_rock, new_wave, post_punk",NaN,1980,156093,0.514,...,9,-10.525,1,0.0487,0.00448,0.00243,0.0594,0.372,158.661,4


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
28221,TRPCPCW128F4269098,Second Skin,The Chameleons,https://p.scdn.co/mp3-preview/3145aa54ba8ea9a1...,0E26LBCuwJKKzt9dZdF0g0,"80s, new_wave, post_punk",Punk,2006,483266,0.446,...,4,-7.817,0,0.134,0.000709,0.203,0.871,0.15,127.917,4


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
13741,TRLHPYP128F4259AB7,Hitsville U.K.,The Clash,https://p.scdn.co/mp3-preview/458093f031b22914...,0e6aQYSW4RjoVf2lSlnuBV,"punk, 80s, punk_rock, new_wave, post_punk, ska",Punk,1991,263200,0.618,...,0,-5.709,1,0.0409,0.0375,0.000003,0.214,0.666,99.807,4


## Matrix Decomposition
for the next part of our recomendation system, we will test using matrix factorization to see if we can produce similar results by finding latent features. To do this, we will need to make a sparse matrix otherwise the algorithm would require too much RAM. We will try different numbers of components as well as Non-negative matrix factorization vs TruncatedSVD and see how each performs.

In [69]:
sparseCombined = csr_matrix(combinedMatrix)

In [70]:
matrixFactor = NMF(n_components=50, init='nndsvd', random_state=42)
W = matrixFactor.fit_transform(sparseCombined)
H = matrixFactor.components_

In [71]:
print("W shape: {}".format(W.shape))
print("H shape: {}".format(H.shape))
# this cell is to show that W is the size of (users, latentFeatures) and H is size (latentFeatures, songs)

W shape: (246177, 50)
H shape: (50, 50683)


In [72]:
def predictFromLatent(userID, n):
    recItems = np.dot(W[id2index[userID]], H)
    retval = np.argpartition(recItems, -n)[-n:]
    return retval

predictions = predictFromLatent('fffff9534445f481b6ab91c345500083d2ce4df1', 4)
showRecomendations(predictions)

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
7042,TRYRZFF128F933BBBB,I Never Told You,Colbie Caillat,https://p.scdn.co/mp3-preview/eecb5e9728c6493a...,0APq1pwGxJFTBw9mHwwnKx,"pop, female_vocalists, singer_songwriter, acou...",Pop,2009,235053,0.501,...,3,-5.195,1,0.029,0.172,0.0,0.13,0.374,144.203,4


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
13741,TRLHPYP128F4259AB7,Hitsville U.K.,The Clash,https://p.scdn.co/mp3-preview/458093f031b22914...,0e6aQYSW4RjoVf2lSlnuBV,"punk, 80s, punk_rock, new_wave, post_punk, ska",Punk,1991,263200,0.618,...,0,-5.709,1,0.0409,0.0375,0.000003,0.214,0.666,99.807,4


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
15267,TRRVJCK12903CD2DCB,U Smile,Justin Bieber,https://p.scdn.co/mp3-preview/b9b8a73b110ac5a4...,0KDJBhhe2OYnnoJtbtXy1f,"pop, hard_rock, black_metal, death_metal, heav...",Pop,2010,196906,0.705,...,1,-5.252,1,0.03,0.516,0.000002,0.117,0.419,112.505,3


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
98,TRFOVTO128F4228CC3,Fortunate Son,Creedence Clearwater Revival,https://p.scdn.co/mp3-preview/3136857b0f683af8...,02W3Xt0zd4cXTQ923Cx9Q3,"rock, classic_rock, 60s",Rock,2005,138052,0.631,...,0,-3.905,1,0.0381,0.0562,0.26,0.201,0.45,132.498,4


In [73]:
matrixFactor2 = NMF(n_components=100, init='nndsvd', random_state = 42)
W = matrixFactor2.fit_transform(sparseCombined)
H = matrixFactor2.components_

predictions = predictFromLatent('fffff9534445f481b6ab91c345500083d2ce4df1', 4)
showRecomendations(predictions)

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
2498,TRMYZFA128F146DE4E,Rollin' & Scratchin',Daft Punk,https://p.scdn.co/mp3-preview/3b6b140e0b84d3c0...,3Wi8fiEg2RkoPaC3PGIQEv,"electronic, dance, 90s, house, techno, electro...",NaN,2006,448626,0.823,...,1,-4.862,1,0.0417,0.0011,0.639,0.0524,0.207,130.038,4


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
16090,TRYEGSH12903CD2DCE,Overboard,Justin Bieber,https://p.scdn.co/mp3-preview/79a899f597b18c3f...,0cfsbkanGUO3yzXCxA0iuF,"pop, black_metal, death_metal, industrial, thr...",NaN,2010,251186,0.48,...,5,-5.214,1,0.0501,0.203,0.0,0.118,0.468,173.907,4


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
38759,TRBZKDX128F4255A4A,Por Besarte,Lu,https://p.scdn.co/mp3-preview/f295fcc4a4baec43...,0JCLAFZZENDUGXVZrV9Mvy,"pop, guitar",Pop,2011,217400,0.443,...,6,-4.433,0,0.0422,0.193,0.0,0.0717,0.525,176.06,4


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
13741,TRLHPYP128F4259AB7,Hitsville U.K.,The Clash,https://p.scdn.co/mp3-preview/458093f031b22914...,0e6aQYSW4RjoVf2lSlnuBV,"punk, 80s, punk_rock, new_wave, post_punk, ska",Punk,1991,263200,0.618,...,0,-5.709,1,0.0409,0.0375,0.000003,0.214,0.666,99.807,4


In [74]:
matrixFactor3 = TruncatedSVD(n_components=100, random_state = 42)
W = matrixFactor2.fit_transform(sparseCombined)
H = matrixFactor2.components_

predictions = predictFromLatent('fffff9534445f481b6ab91c345500083d2ce4df1', 4)
showRecomendations(predictions)

,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
2498,TRMYZFA128F146DE4E,Rollin' & Scratchin',Daft Punk,https://p.scdn.co/mp3-preview/3b6b140e0b84d3c0...,3Wi8fiEg2RkoPaC3PGIQEv,"electronic, dance, 90s, house, techno, electro...",NaN,2006,448626,0.823,...,1,-4.862,1,0.0417,0.0011,0.639,0.0524,0.207,130.038,4


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
16090,TRYEGSH12903CD2DCE,Overboard,Justin Bieber,https://p.scdn.co/mp3-preview/79a899f597b18c3f...,0cfsbkanGUO3yzXCxA0iuF,"pop, black_metal, death_metal, industrial, thr...",NaN,2010,251186,0.48,...,5,-5.214,1,0.0501,0.203,0.0,0.118,0.468,173.907,4


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
38759,TRBZKDX128F4255A4A,Por Besarte,Lu,https://p.scdn.co/mp3-preview/f295fcc4a4baec43...,0JCLAFZZENDUGXVZrV9Mvy,"pop, guitar",Pop,2011,217400,0.443,...,6,-4.433,0,0.0422,0.193,0.0,0.0717,0.525,176.06,4


,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
13741,TRLHPYP128F4259AB7,Hitsville U.K.,The Clash,https://p.scdn.co/mp3-preview/458093f031b22914...,0e6aQYSW4RjoVf2lSlnuBV,"punk, 80s, punk_rock, new_wave, post_punk, ska",Punk,1991,263200,0.618,...,0,-5.709,1,0.0409,0.0375,0.000003,0.214,0.666,99.807,4


# Results and Analysis
It appears that in every model, when checking the recomended songs for a user who listens to "Hitsville U.K." the most, the same song is always recomended, however in the Song-Song similarity matrix approach, it would recomend other songs that had similar tags to the most listened to song whereas in the Matrix Decomposition method it was finding latent features between songs that I would not have thought of as similar.

The Matrix Factorization is not necessarily a bad model, but it is hard to interpret since the latent factors are not known to us and are instead inferred.

The Song-Song similarity matrix however has an easily understandable approach to how to define similarity between songs and allows us to be able to tweak the matrix easier as we learn more from the user. Since there is not a defined right answer to this recomendation approach, I would use the Song-Song similarity model. This is because we can tweak the similarities as users provide either negative feedback by skipping the songs we recomend or positive feedback by listening to the songs we recomend.

# Discussion

## Takeaways
It appears from a visual inspection that most of the songs recomended by matrix factorization are not similar to Hitsville U.K and provide mostly vastly different results. This could imply that other people that listen to hitsville U.K also listen to the other music and that there are similarities between these songs that we can not see directly. Our earlier algorithm though with similarities between songs is intuitive and we can see that the songs appear to sound similar by subjective inspection.

## Limitations
One major limitation is the amount of memory available on my computer. I had to create workarounds by removing users from the dataframe and storing the song-song similarity matrix on my disk instead of accessing it in RAM. Another limitation is the description of the songs. Ideally we could have each column be more meaningful than the representation we used for each song and this inturn would provide us with a more meaningful and accurate similarity matrix. Lastly, we do not have a meaningful metric to score whether our recomendations to the user are accurate or not. For other item-item similarity matrices we have computed a score of how different it is from our expectation (observed values) but in this case the data is so sparse and we would never recomend new songs in that case.

## Future
In the future, I think it would be good to bring on an expert who has studied music theory in order to get a more accurate similarity measure between songs. I do not believe that the cosine and jaccard similarity that we used is the best measure and that there is room for improvement. I also would like to be able to move the recomendations into production and see which model appears to do the best by returning positive feedback for recomended songs listened to and negative feedback for recomended songs skipped.

A way to verify if our models are working correctly would be to attempt to reccomend the songs to users. If they skip the song then they do not like it and our algorithm incorrectly reccomended a song. If they decide to listen to the song or even relisten to it, then our algorithm is working properly. We can take the feedback that we learn from this approach to fine tune our model further, however in this case it was static data obtained through kaggle and therefore we can not measure our model's effectiveness.

Lastly, the algorithm could be fine tuned by changing the song similarity metrics used. In our case we simply used a cosine similarity function and jaccard similarity function, however there might be a better way to measure weighted distances between certain columns that someone who studies music might know that would allow us a better performing algorithm.

# Citations

Million Song Dataset + Spotify + Last.fm. (n.d.). Www.kaggle.com. https://www.kaggle.com/datasets/undefinenull/million-song-dataset-spotify-lastfm/data

‌